## too slow to run local - moving to google colab

In [23]:
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
import numpy.linalg as npl
np.set_printoptions(precision=2)

N = 35
M = 28*28
X = np.empty((N,M))
for i in range(N):
    path = "./data/ucla/ucla-%d"%(i*10)
    X[i] = plt.imread(path).mean(2).reshape(M)

In [26]:
# generate ordering set
D = np.array([[npl.norm(X[i]-X[j]) for i in range(N)] for j in range(N)])

imap = {} # map array index to matrix index
i = 0
d = np.array([])
for r in range(N):
    for c in range(r+1, N):
        imap[i] = (r, c)
        d = np.append(d, D[r,c])
        i += 1
        
idx = np.argsort(d)
d = d[idx]
d, unique_idx = np.unique(d, return_index=True)
idx = idx[unique_idx]
# print(d)

# full order : len(d)C2 comparisons
S = [imap[idx[i]] + imap[idx[j]] for i in range(len(idx)) for j in range(i+1,len(idx))] 

In [28]:
# inequality constraints
A = []
for (i,j,k,l) in S:
    a = np.zeros((N,N))
    a[k,k] += 1
    a[k,l] += -2
    a[l,l] += 1
    a[i,i] += -1
    a[i,j] += 2
    a[j,j] += -1
    A.append(a)
    
# problem
K_ = cp.Variable((N,N), symmetric=True) # gram matrix
E = cp.Variable((len(S),1), nonneg=True) # slack

diff = 1 # force difference between comparisons
gamma = 0.22 # reg param for rank

constraints = []
constraints += [K_ >> 0] # PSD
constraints += [cp.sum(K_) == 0] # center
constraints += [cp.trace(A[e]@K_) >= (diff - E[e]) for e in range(len(S))]

error = cp.sum(E)
obj = cp.Minimize(error + gamma*cp.trace(K_))
prob = cp.Problem(obj, constraints)
# prob.solve()

In [29]:
%time prob.solve()

CPU times: user 2min 49s, sys: 4.61 s, total: 2min 53s
Wall time: 2min 46s


/usr/local/lib/python3.8/site-packages/cvxpy/problems/problem.py:1267: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


340.63884964680375

In [30]:
K = X@X.T
U, V = npl.eigh(K_.value)
print(npl.eigh(K)[0])
print(npl.eigh(K_.value)[0])

[1.12e+00 1.12e+00 1.55e+00 1.59e+00 1.67e+00 1.69e+00 1.69e+00 1.75e+00
 1.93e+00 2.03e+00 2.11e+00 2.51e+00 2.57e+00 2.73e+00 2.77e+00 2.77e+00
 2.90e+00 3.19e+00 3.38e+00 3.59e+00 4.45e+00 4.72e+00 5.65e+00 6.95e+00
 7.80e+00 7.83e+00 8.36e+00 9.20e+00 1.50e+01 1.60e+01 1.65e+01 1.72e+01
 2.44e+01 2.76e+01 2.59e+04]
[-9.54e-05 -1.20e-05 -4.79e-06 -4.29e-06 -3.87e-06 -2.98e-06 -1.29e-06
 -1.03e-06 -1.48e-10  2.82e-08  6.20e-07  1.65e-06  2.21e-06  2.97e-06
  3.88e-06  4.72e-06  4.85e-06  6.29e-06  7.81e-06  1.15e-05  1.39e-05
  2.44e-05  3.78e-05  1.87e+00  1.36e+01  1.93e+01  2.69e+01  3.40e+01
  5.72e+01  6.49e+01  8.42e+01  8.97e+01  1.23e+02  1.67e+02  2.23e+02]
